In [1]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import plotly.express as px


In [2]:
supply = pd.read_excel(r"C:\Users\parke\Desktop\drivers_acceptance_price.csv")
supply = supply.rename(columns = str.lower)

supply.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pay       1000 non-null   float64
 1   accepted  1000 non-null   int64  
dtypes: float64(1), int64(1)
memory usage: 15.8 KB


In [ ]:
px.scatter(supply, x = 'pay', y = 'accepted')

In [3]:
logit = smf.logit("accepted ~ pay", supply).fit()
logit.summary()

Optimization terminated successfully.
         Current function value: 0.394998
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               accepted   No. Observations:                 1000
Model:                          Logit   Df Residuals:                      998
Method:                           MLE   Df Model:                            1
Date:                Wed, 30 Nov 2022   Pseudo R-squ.:                  0.4289
Time:                        23:53:11   Log-Likelihood:                -395.00
converged:                       True   LL-Null:                       -691.69
Covariance Type:            nonrobust   LLR p-value:                4.606e-131
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -6.3158      0.409    -15.455      0.000      -7.117      -5.515
pay            0.2539      0.016     15.979      0.000       0.223       0.285
==============================================================================
"""

In [ ]:
# Generate a dataframe of all potential riders for each month
rider = pd.Series(data = np.arange(start = 0, stop =rider_pool), name = 'rider')
month = pd.Series(data = np.arange(start = 0, stop = num_periods), name = 'month')

data = pd.merge(rider, month, how = 'cross')
data = data.assign(acquired = np.NaN,
                    acquisition_month = np.NaN,
                    lam = 0,
                    requests = np.NaN,
                    matches = np.NaN
                )
data.head()

# 1 period per month, 12 months
num_periods =10
period = np.arange(0,num_periods)

# Additional parameters
rider_pool = 10000
acquisition_cost = 30

# Coarse grid of acquisitions and rider payments
acquisition_grid = [200,400,600,800,1000]
pay_grid = [0,10,20,30,40,50]


In [ ]:
# 1 period per month, 12 months
num_periods =10
period = np.arange(0,num_periods)

# Additional parameters
rider_pool = 10000
acquisition_cost = 30

# Coarse grid of acquisitions and rider payments
acquisition_grid = [200,400,600,800,1000]
pay_grid = [0,10,20,30,40,50]



In [74]:
# 1 period per month, 12 months
num_periods =10
period = np.arange(0,num_periods)

# Additional parameters
rider_pool = 10000
acquisition_cost = 30
ride_cost = 30

# Coarse grid of acquisitions and rider payments
acquisition_grid = [200,400,600,800,1000]
pay_grid = [15,20,25,30,35]

# Generate a dataframe of all potential riders for each month
rider = pd.Series(data = np.arange(start = 0, stop =rider_pool), name = 'rider')
month = pd.Series(data = np.arange(start = 0, stop = num_periods), name = 'month')

data = pd.merge(rider, month, how = 'cross')
data = data.assign(acquired = 0,
                    acquisition_month = np.NaN,
                    lam = 0,
                    requests = 0,
                    matches = 0
                )

non_acquired = np.arange(0, rider_pool)
acquired_rider_list = np.array([])

acquisition_list = np.array([])
pay_list = np.array([])


for t in period:

    # draw a number of acquisitions and driver payout for the period
    acquisitions = np.random.choice(acquisition_grid, size = 1)
    acquisition_list = np.concatenate((acquisition_list,acquisitions))

    pay = pd.Series(np.random.choice(pay_grid, size= 1), name = 'pay')
    pay_list = np.concatenate((pay_list,pay))

    # determine the probability that a ride is matched given the payout
    match_prob = logit.predict(pay).iat[0]

    # select the riders to be acquired this period and remove them from the non_acquired list
    acquired = np.random.choice(non_acquired, size = acquisitions, replace = False)
    non_acquired = np.setdiff1d(non_acquired, acquired)
    acquired_rider_list = np.concatenate((acquired_rider_list, acquired))

    # update the relevant columns for the newly acquired
    data.loc[(data['rider'].isin(acquired)) & (data['month'] >= t), 'acquired'] = 1
    data.loc[data['rider'].isin(acquired), 'acquisition_month'] = t
    data.loc[(data['rider'].isin(acquired)) & (data['month'] == t), 'lam'] = 1
    #data.loc[(data['rider'].isin(acquired)) & (data['month'] == t), 'acq_cost'] = acquisition_cost


    # update lambda for the already acquired
    data.loc[(data['month'] == t) & (data['acquired'] == 1) & (data['acquisition_month'] < t) & (data['month'] != 0), 'lam'] = data.groupby(['rider'])['matches'].shift(1)

    # compute ride requests and successful matches for all acquired riders
    filters = (data['rider'].isin(acquired_rider_list)) & (data['month'] == t)
    
    data.loc[filters, 'requests'] = data['lam'].apply(lambda x: np.random.poisson(x, size = 1)[0])
    data.loc[filters, 'matches'] = data['requests'].apply(lambda x: np.round(match_prob*x))

# save out the pay and acquisition vectors
pay_series = pd.Series(data = pay_list, name= 'pay')
acquistion_series = pd.Series(data = acquisition_list, name = 'acquisitions')

# Add in final useful columns
df = data.merge(pay_series, how = 'left', left_on='month', right_index= True)
df['rider_spend'] = df['matches'].multiply(ride_cost)
df['driver_pay'] = df['matches']*df['pay']
df['acquisition_cost'] = 0
df.loc[df['acquisition_month'] == df['month'], 'acquisition_cost'] = 30
df['profit'] = df['rider_spend'] - df['driver_pay'] - df['acquisition_cost']

summary = df[['requests','matches','rider_spend','driver_pay','acquisition_cost','profit']].sum()
summary_monthly = df.groupby('month', as_index = False)[['requests','matches','rider_spend','driver_pay','acquisition_cost','profit']].sum()




In [75]:
summary_monthly

,month,requests,matches,rider_spend,driver_pay,acquisition_cost,profit
0,0,191,142,4260,3550.0,6000,-5290.0
1,1,545,507,15210,15210.0,12000,-12000.0
2,2,928,0,0,0.0,12000,-12000.0
3,3,405,364,10920,10920.0,12000,-12000.0
4,4,1129,0,0,0.0,24000,-24000.0
5,5,845,75,2250,1500.0,24000,-23250.0
6,6,467,0,0,0.0,12000,-12000.0
7,7,381,0,0,0.0,12000,-12000.0
8,8,173,173,5190,6055.0,6000,-6865.0
9,9,972,880,26400,26400.0,24000,-24000.0


In [80]:
# 1 period per month, 12 months
num_periods =10
period = np.arange(0,num_periods)

# Additional parameters
rider_pool = 10000
acquisition_cost = 30
ride_cost = 30

# Coarse grid of acquisitions and rider payments
acquisition_grid = [200,400,600,800,1000]
pay_grid = [15,20,25,30,35]

overall_summaries = []
monthly_summaries = []
pay_vectors = []
acquisition_vectors = []

for i in range(30):


    # Generate a dataframe of all potential riders for each month
    rider = pd.Series(data = np.arange(start = 0, stop =rider_pool), name = 'rider')
    month = pd.Series(data = np.arange(start = 0, stop = num_periods), name = 'month')

    data = pd.merge(rider, month, how = 'cross')
    data = data.assign(acquired = 0,
                        acquisition_month = np.NaN,
                        lam = 0,
                        requests = 0,
                        matches = 0
                    )

    non_acquired = np.arange(0, rider_pool)
    acquired_rider_list = np.array([])

    acquisition_list = np.array([])
    pay_list = np.array([])


    for t in period:

        # draw a number of acquisitions and driver payout for the period
        acquisitions = np.random.choice(acquisition_grid, size = 1)
        acquisition_list = np.concatenate((acquisition_list,acquisitions))

        pay = pd.Series(np.random.choice(pay_grid, size= 1), name = 'pay')
        pay_list = np.concatenate((pay_list,pay))

        # determine the probability that a ride is matched given the payout
        match_prob = logit.predict(pay).iat[0]

        # select the riders to be acquired this period and remove them from the non_acquired list
        acquired = np.random.choice(non_acquired, size = acquisitions, replace = False)
        non_acquired = np.setdiff1d(non_acquired, acquired)
        acquired_rider_list = np.concatenate((acquired_rider_list, acquired))

        # update the relevant columns for the newly acquired
        data.loc[(data['rider'].isin(acquired)) & (data['month'] >= t), 'acquired'] = 1
        data.loc[data['rider'].isin(acquired), 'acquisition_month'] = t
        data.loc[(data['rider'].isin(acquired)) & (data['month'] == t), 'lam'] = 1
        #data.loc[(data['rider'].isin(acquired)) & (data['month'] == t), 'acq_cost'] = acquisition_cost


        # update lambda for the already acquired
        data.loc[(data['month'] == t) & (data['acquired'] == 1) & (data['acquisition_month'] < t) & (data['month'] != 0), 'lam'] = data.groupby(['rider'])['matches'].shift(1)

        # compute ride requests and successful matches for all acquired riders
        filters = (data['rider'].isin(acquired_rider_list)) & (data['month'] == t)
        
        data.loc[filters, 'requests'] = data['lam'].apply(lambda x: np.random.poisson(x, size = 1)[0])
        data.loc[filters, 'matches'] = data['requests'].apply(lambda x: np.round(match_prob*x))

    # save out the pay and acquisition vectors
    pay_series = pd.Series(data = pay_list, name= 'pay')
    acquisition_series = pd.Series(data = acquisition_list, name = 'acquisitions')

    # Add in final useful columns
    df = data.merge(pay_series, how = 'left', left_on='month', right_index= True)
    df['rider_spend'] = df['matches'].multiply(ride_cost)
    df['driver_pay'] = df['matches']*df['pay']
    df['acquisition_cost'] = 0
    df.loc[df['acquisition_month'] == df['month'], 'acquisition_cost'] = 30
    df['profit'] = df['rider_spend'] - df['driver_pay'] - df['acquisition_cost']

    summary = df[['requests','matches','rider_spend','driver_pay','acquisition_cost','profit']].sum()
    monthly_summary = df.groupby('month', as_index = False)[['requests','matches','rider_spend','driver_pay','acquisition_cost','profit']].sum()

    overall_summaries.append(summary)
    monthly_summaries.append(monthly_summary)
    pay_vectors.append(pay_series)
    acquisition_vectors.append(acquisition_series)




In [ ]:
pv_df  = pd.DataFrame(data = pay_vectors).reset_index()
ac_df  = pd.DataFrame(data = acquisition_vectors).reset_index()
s_df  = pd.DataFrame(data = overall_summaries).reset_index()

